In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
auth.authenticate_user()

In [ ]:
client = bigquery.Client(project = "continual-apex-402217")

# Vínculo Trabalhista vs. Saúde

## Entre as pessoas que foram entrevistadas, qual a quantidade que trabalhou ou fez algum bico (por pelo menos uma hora) na semana passada?

In [ ]:
query = """
SELECT
  `Mes_Semana`, `Trabalhou_ou_fez_algum_bico_por_pelo_menos_uma_hora_na_semana_passada`,
  COUNT(*) AS `Quantidade`
FROM
  continual-apex-402217.covid.pnad_covid
WHERE
  `Trabalhou_ou_fez_algum_bico_por_pelo_menos_uma_hora_na_semana_passada` IS NOT NULL
GROUP BY
  `Mes_Semana`, `Trabalhou_ou_fez_algum_bico_por_pelo_menos_uma_hora_na_semana_passada`
"""

query_job = client.query(query)

df_trabalhou_semana_passada = query_job.to_dataframe()

df_trabalhou_semana_passada.head(20)

,Mes_Semana,Trabalhou_ou_fez_algum_bico_por_pelo_menos_uma_hora_na_semana_passada,Quantidade
0,5/1,Não,166624
1,5/1,Sim,100920
2,6/1,Sim,118852
3,6/1,Não,184672
4,7/1,Não,183552
5,7/1,Sim,126072
6,5/2,Sim,109172
7,5/2,Não,178736
8,6/2,Sim,125056
9,6/2,Não,192812


In [ ]:
df_trabalhou_semana_passada.to_csv('df_trabalhou_semana_passada.csv', index=False)

In [ ]:
fig = px.bar(df_trabalhou_semana_passada,
             x = 'Mes_Semana',
             y = 'Quantidade',
             color = 'Trabalhou_ou_fez_algum_bico_por_pelo_menos_uma_hora_na_semana_passada',
             barmode = 'group',  # Define o modo para 'group' para barras lado a lado
             labels = {'Trabalhou_ou_fez_algum_bico_por_pelo_menos_uma_hora_na_semana_passada': 'Trabalhou ou fez algum bico (por pelo menos uma hora) na semana passada', 'Mes_Semana': 'Mês/Semana'},
             title = 'Quantidade de pessoas que trabalhou ou fez algum bico (por pelo menos uma hora) na semana passada'
             )
fig.update_traces(texttemplate = '%{y:,.2s}', textposition = 'outside')  # Define o rótulo como a Quantidade (y) e a posição como 'outside'
fig.show()

## Dentre esses que trabalharam ou fizeram algum bico (por pelo menos uma hora) na semana passada, quantos deles apresentaram sintomas?

In [ ]:
query = """
SELECT
  `Mes_Semana`,
  CASE
    WHEN (`Febre` = 'Sim' OR `Tosse` = 'Sim' OR `Dor_de_garganta` = 'Sim' OR `Dificuldade_para_respirar` = 'Sim' OR `Dor_de_cabeca` = 'Sim' OR `Perda_de_cheiro_ou_sabor` = 'Sim' OR `Dor_muscular` = 'Sim') THEN 'Sim'
    ELSE 'Não'
  END AS `Teve_Sintomas`,
  -- `Tipo_de_vinculo_trabalhista`,
  COUNT(*) AS `Quantidade`
FROM
  continual-apex-402217.covid.pnad_covid
WHERE
  `Trabalhou_ou_fez_algum_bico_por_pelo_menos_uma_hora_na_semana_passada` = 'Sim'
GROUP BY
   `Mes_Semana`,
  CASE
    WHEN (`Febre` = 'Sim' OR `Tosse` = 'Sim' OR `Dor_de_garganta` = 'Sim' OR `Dificuldade_para_respirar` = 'Sim' OR `Dor_de_cabeca` = 'Sim' OR `Perda_de_cheiro_ou_sabor` = 'Sim' OR `Dor_muscular` = 'Sim') THEN 'Sim'
    ELSE 'Não'
  END,
  `Teve_Sintomas`  --, `Tipo_de_vinculo_trabalhista`
ORDER BY
  `Mes_Semana` ASC
"""

query_job = client.query(query)

df_sintomas = query_job.to_dataframe()

df_sintomas.head(10)


,Mes_Semana,Teve_Sintomas,Quantidade
0,5/1,Não,89088
1,5/1,Sim,11832
2,5/2,Sim,11468
3,5/2,Não,97704
4,5/3,Não,101260
5,5/3,Sim,11312
6,5/4,Não,107084
7,5/4,Sim,11092
8,6/1,Não,110568
9,6/1,Sim,8284


In [ ]:
df_sintomas.to_csv('df_sintomas.csv', index=False)

In [ ]:
fig = px.bar(df_sintomas,
             x = 'Mes_Semana',
             y = 'Quantidade',
             color = 'Teve_Sintomas',
             barmode = 'group',  # Define o modo para 'group' para barras lado a lado
             labels = {'Teve_Sintomas': 'Teve sintomas?', 'Mes_Semana': 'Mês/Semana'},
             title = 'Quantidade de pessoas que apresentaram algum sintoma entre as pessoas que trabalharam ou fizeram algum bico (por pelo menos uma hora) na semana passada'
             )
fig.update_traces(texttemplate = '%{y:,.2s}', textposition = 'outside')  # Define o rótulo como a Quantidade (y) e a posição como 'outside'
fig.show()

## E desses, quais os tipos de vínculos trabalhistas mais comuns?

In [ ]:
query = """
SELECT
  `Tipo_de_vinculo_trabalhista`, COUNT(*) AS Quantidade
FROM
  continual-apex-402217.covid.pnad_covid
WHERE
  `Trabalhou_ou_fez_algum_bico_por_pelo_menos_uma_hora_na_semana_passada` = 'Sim'
  AND
  (
    `Febre` = 'Sim'
    OR `Tosse` = 'Sim'
    OR `Dor_de_garganta` = 'Sim'
    OR `Dificuldade_para_respirar` = 'Sim'
    OR `Dor_de_cabeca` = 'Sim'
    OR `Perda_de_cheiro_ou_sabor` = 'Sim'
    OR `Dor_muscular` = 'Sim'
  )
GROUP BY `Tipo_de_vinculo_trabalhista`
ORDER BY `Quantidade` ASC
"""

query_job = client.query(query)

df_vinculo = query_job.to_dataframe()

df_vinculo.head(20)

,Tipo_de_vinculo_trabalhista,Quantidade
0,"Militar do exercito, marinha ou aeronáutica",384
1,Policial militar ou bombeiro militar,600
2,Trabalhador familiar não remunerado em ajuda a...,1916
3,Estava fora do mercado de trabalho (fazia apen...,2996
4,Empregador,3432
5,"Trabalhador doméstico (empregado doméstico, cu...",6420
6,Empregado do setor público (inclusive empresas...,17976
7,Conta própria,32316
8,Empregado do setor privado,45020


In [ ]:
df_vinculo.to_csv('df_vinculo.csv', index=False)

In [ ]:
fig = px.bar(df_vinculo,
             x = 'Tipo_de_vinculo_trabalhista',
             y = 'Quantidade',
             title = 'Tipos de vínculo trabalhistas',
             height = 900
             )
fig.update_xaxes(tickangle = 30)  # Ajusta o ângulo dos rótulos no eixo X

fig.update_traces(texttemplate = '%{y}', textposition = 'outside')  # Define o rótulo como a Quantidade (y) e a posição como 'outside'
fig.show()

## Ainda entre esses que apresentaram sintomas, quais foram os sintomas mais comuns?

In [ ]:
query = """
SELECT
  COUNT(CASE WHEN `Febre` = 'Sim' THEN 1 ELSE NULL END) AS `Febre`,
  COUNT(CASE WHEN `Tosse` = 'Sim' THEN 1 ELSE NULL END) AS `Tosse`,
  COUNT(CASE WHEN `Dor_de_garganta` = 'Sim' THEN 1 ELSE NULL END) AS `Dor_de_garganta`,
  COUNT(CASE WHEN `Dificuldade_para_respirar` = 'Sim' THEN 1 ELSE NULL END) AS `Dificuldade_para_respirar`,
  COUNT(CASE WHEN `Dor_de_cabeca` = 'Sim' THEN 1 ELSE NULL END) AS `Dor_de_cabeca`,
  COUNT(CASE WHEN `Perda_de_cheiro_ou_sabor` = 'Sim' THEN 1 ELSE NULL END) AS `Perda_de_cheiro_ou_sabor`,
  COUNT(CASE WHEN `Dor_muscular` = 'Sim' THEN 1 ELSE NULL END) AS `Dor_muscular`
FROM
  continual-apex-402217.covid.pnad_covid
WHERE
  `Trabalhou_ou_fez_algum_bico_por_pelo_menos_uma_hora_na_semana_passada` = 'Sim'
  AND
  (
    `Febre` = 'Sim'
    OR `Tosse` = 'Sim'
    OR `Dor_de_garganta` = 'Sim'
    OR `Dificuldade_para_respirar` = 'Sim'
    OR `Dor_de_cabeca` = 'Sim'
    OR `Perda_de_cheiro_ou_sabor` = 'Sim'
    OR `Dor_muscular` = 'Sim'
  )
"""

query_job = client.query(query)

df_sintomas_comuns = query_job.to_dataframe()

df_sintomas_comuns.head(20)

,Febre,Tosse,Dor_de_garganta,Dificuldade_para_respirar,Dor_de_cabeca,Perda_de_cheiro_ou_sabor,Dor_muscular
0,22492,35708,30244,14172,65384,15888,34928


In [ ]:
df_sintomas_comuns = df_sintomas_comuns.T.reset_index()
df_sintomas_comuns.columns = ['Sintomas', 'Quantidade']

sintomas_formatados = {
    'Febre': 'Febre',
    'Tosse': 'Tosse',
    'Dor_de_garganta': 'Dor de garganta',
    'Dificuldade_para_respirar': 'Dificuldade para respirar',
    'Dor_de_cabeca': 'Dor de cabeça',
    'Perda_de_cheiro_ou_sabor': 'Perda de cheiro ou sabor',
    'Dor_muscular': 'Dor muscular'
}

# Aplicar o mapeamento para os nomes das colunas
df_sintomas_comuns['Sintomas'] = df_sintomas_comuns['Sintomas'].map(sintomas_formatados)

In [ ]:
df_sintomas_comuns.to_csv('df_sintomas_comuns.csv', index=False)

In [ ]:
fig = px.bar(df_sintomas_comuns,
             x = 'Sintomas',
             y = 'Quantidade',
             title = 'Quantidade de pessoas que apresentaram cada sintoma'
             )
fig.update_traces(texttemplate = '%{y}', textposition = 'outside')  # Define o rótulo como a Quantidade (y) e a posição como 'outside'
fig.show()

## E qual a atitude que essas pessoas tomaram?

In [ ]:
query = """
SELECT
  `Mes_Semana`, `Apenas_ficou_em_casa` AS Casa, COUNT(*) AS Quantidade
FROM
  continual-apex-402217.covid.pnad_covid
WHERE
  `Apenas_ficou_em_casa` IS NOT NULL
  AND `Apenas_ficou_em_casa` <> 'Ignorado'
  AND
  (
    `Febre` = 'Sim'
    OR `Tosse` = 'Sim'
    OR `Dor_de_garganta` = 'Sim'
    OR `Dificuldade_para_respirar` = 'Sim'
    OR `Dor_de_cabeca` = 'Sim'
    OR `Perda_de_cheiro_ou_sabor` = 'Sim'
    OR `Dor_muscular` = 'Sim'
  )
GROUP BY
   `Mes_Semana`, `Apenas_ficou_em_casa`
ORDER BY
  `Mes_Semana` ASC
"""

query_job = client.query(query)

df_atitude_tomada = query_job.to_dataframe()

df_atitude_tomada.head(20)

,Mes_Semana,Casa,Quantidade
0,5/1,Sim,23820
1,5/1,Não,4696
2,5/2,Não,4388
3,5/2,Sim,23560
4,5/3,Sim,22852
5,5/3,Não,3996
6,5/4,Sim,21332
7,5/4,Não,3992
8,6/1,Sim,15708
9,6/1,Não,3008


In [ ]:
df_atitude_tomada.to_csv('df_atitude_tomada.csv', index=False)

In [ ]:
fig = px.bar(df_atitude_tomada,
             x = 'Mes_Semana',
             y = 'Quantidade',
             color = 'Casa',
             barmode = 'relative',  # Define o modo para 'group' para barras lado a lado
             labels = {'Mes_Semana': 'Mês/Semana', 'Casa': 'Apenas ficou em casa?'},
             title = 'Essa pessoa que apresentou sintomas apenas ficou em casa?'
             )
fig.update_traces(texttemplate = '%{y}', textposition = 'outside')  # Define o rótulo como a Quantidade (y) e a posição como 'outside'
fig.show()

## Caso a pessoa não tenha apenas ficado em casa, ou seja, tenha procurado ajuda médica, ela comprou e/ou tomou remédios por orientação médica?

In [ ]:
query = """
SELECT
  `Mes_Semana`, `Comprou_e_ou_tomou_remedio_por_orientacao_medica` AS Remedio, COUNT(*) AS Quantidade
FROM
  continual-apex-402217.covid.pnad_covid
WHERE
  `Apenas_ficou_em_casa` = 'Não'
  AND `Comprou_e_ou_tomou_remedio_por_orientacao_medica` <> 'Ignorado'
  AND
  (
    `Febre` = 'Sim'
    OR `Tosse` = 'Sim'
    OR `Dor_de_garganta` = 'Sim'
    OR `Dificuldade_para_respirar` = 'Sim'
    OR `Dor_de_cabeca` = 'Sim'
    OR `Perda_de_cheiro_ou_sabor` = 'Sim'
    OR `Dor_muscular` = 'Sim'
  )
GROUP BY
   `Mes_Semana`, `Comprou_e_ou_tomou_remedio_por_orientacao_medica`
ORDER BY
  `Mes_Semana`, `Comprou_e_ou_tomou_remedio_por_orientacao_medica`
"""

query_job = client.query(query)

df_remedio = query_job.to_dataframe()

df_remedio.head(20)

,Mes_Semana,Remedio,Quantidade
0,5/1,Não,4108
1,5/1,Sim,580
2,5/2,Não,3912
3,5/2,Sim,476
4,5/3,Não,3380
5,5/3,Sim,608
6,5/4,Não,3480
7,5/4,Sim,504
8,6/1,Não,2620
9,6/1,Sim,388


In [ ]:
df_remedio.to_csv('df_remedio.csv', index=False)

In [ ]:
fig = px.bar(df_remedio,
             x = 'Mes_Semana',
             y = 'Quantidade',
             color = 'Remedio',
             barmode = 'relative',  # Define o modo para 'group' para barras lado a lado
             labels = {'Mes_Semana': 'Mês/Semana', 'Remedio': 'Comprou e/ou tomou remédios por orientação médica?'},
             title = 'Quantidade de pessoas que foi a alguma unidade de saúde e tomou ou não remédios por orientação médica'
             )
fig.update_traces(texttemplate = '%{y}', textposition = 'outside')  # Define o rótulo como a Quantidade (y) e a posição como 'outside'
fig.show()

## E das pessoas que procuraram alguma unidade de saúde e foram internadas por um dia ou mais, quantas foram sedadas, entubadas ou colocadas em respiração artificial com ventilador?

In [ ]:
query = """
SELECT
  `Mes_Semana`, `Sedado_e_entubado_e_colocado_em_respiracao_artificial_com_ventilador` AS `Tratamento`,
  COUNT(*) AS Quantidade
FROM
  continual-apex-402217.covid.pnad_covid
WHERE
  (
    `Foi_a_algum_estabelecimento_de_saude` = 'Sim'
    OR `Foi_em_Posto_ou_UBS_ou_ESF` = 'Sim'
    OR `Foi_em_pronto_socorro_do_SUS_ou_UPA` = 'Sim'
    OR `Foi_em_hospital_do_SUS` = 'Sim'
  )
  AND `Internado_por_um_dia_ou_mais` = 'Sim'
  AND `Sedado_e_entubado_e_colocado_em_respiracao_artificial_com_ventilador` IS NOT NULL
  AND `Sedado_e_entubado_e_colocado_em_respiracao_artificial_com_ventilador` <> 'Ignorado'
GROUP BY
  `Mes_Semana`, `Sedado_e_entubado_e_colocado_em_respiracao_artificial_com_ventilador`
ORDER BY
  `Mes_Semana`, `Sedado_e_entubado_e_colocado_em_respiracao_artificial_com_ventilador`
"""

query_job = client.query(query)

df_entubacao = query_job.to_dataframe()

df_entubacao.head(20)

,Mes_Semana,Tratamento,Quantidade
0,5/1,Não,108
1,5/1,Sim,44
2,5/2,Não,128
3,5/2,Sim,56
4,5/3,Não,164
5,5/3,Sim,40
6,5/4,Não,200
7,5/4,Sim,60
8,6/1,Não,164
9,6/1,Sim,80


In [ ]:
df_entubacao.to_csv('df_entubacao.csv', index=False)

In [ ]:
fig = px.bar(df_entubacao,
             x = 'Mes_Semana',
             y = 'Quantidade',
             color = 'Tratamento',
             barmode = 'relative',  # Define o modo para 'group' para barras lado a lado
             labels = {'Mes_Semana': 'Mês/Semana', 'Tratamento': 'A pessoa foi sedada, entubada ou colocada em respiração artificial com ventilador'},
             title = 'Quantidade de pessoas que procuraram alguma unidade de saúde, foram internadas por um dia ou mais e foram sedadas, entubadas ou colocadas em respiração artificial com ventilador'
             )
fig.update_traces(texttemplate = '%{y}', textposition = 'outside')  # Define o rótulo como a Quantidade (y) e a posição como 'outside'
fig.show()

## E dessas pessoas que foram sedadas, entubadas ou colocadas em respiração artificial com ventilador, quantas possuem plano de saúde?

In [ ]:
query = """
SELECT
  `Mes_Semana`, `Plano_de_saude`,
  COUNT(*) AS Quantidade
FROM
  continual-apex-402217.covid.pnad_covid
WHERE
  (
    `Foi_a_algum_estabelecimento_de_saude` = 'Sim'
    OR `Foi_em_Posto_ou_UBS_ou_ESF` = 'Sim'
    OR `Foi_em_pronto_socorro_do_SUS_ou_UPA` = 'Sim'
    OR `Foi_em_hospital_do_SUS` = 'Sim'
  )
  AND `Internado_por_um_dia_ou_mais` = 'Sim'
  AND `Sedado_e_entubado_e_colocado_em_respiracao_artificial_com_ventilador` = 'Sim'
  AND `Plano_de_saude` <> 'Ignorado'
GROUP BY
   `Mes_Semana`, `Plano_de_saude`
ORDER BY
  `Mes_Semana`, `Plano_de_saude`
"""

query_job = client.query(query)

df_plano_saude = query_job.to_dataframe()

df_plano_saude.head(20)

,Mes_Semana,Plano_de_saude,Quantidade
0,5/1,Não,24
1,5/1,Sim,20
2,5/2,Não,40
3,5/2,Sim,16
4,5/3,Não,32
5,5/3,Sim,8
6,5/4,Não,52
7,5/4,Sim,8
8,6/1,Não,52
9,6/1,Sim,28


In [ ]:
df_plano_saude.to_csv('df_plano_saude.csv', index=False)

In [ ]:
fig = px.bar(df_plano_saude,
             x = 'Mes_Semana',
             y = 'Quantidade',
             color = 'Plano_de_saude',
             barmode = 'relative',  # Define o modo para 'group' para barras lado a lado
             labels = {'Mes_Semana': 'Mês/Semana', 'Plano_de_saude': 'Tem plano de saúde?'},
             title = 'Quantidade de pessoas que possuem plano de saúde dentre as que foram internadas por um dia ou mais e foram sedadas, entubadas ou colocadas em respiração artificial com ventilador'
             )
fig.update_traces(texttemplate = '%{y}', textposition = 'outside')  # Define o rótulo como a Quantidade (y) e a posição como 'outside'
fig.show()

# Adoção do Trabalho Remoto

In [ ]:
query_cenario_pandemia = """
SELECT
  mes_semana,
  SUM(casos) AS casos,
  SUM(obitos) AS obitos
FROM
  continual-apex-402217.covid.datasus_covid
GROUP BY mes_semana
"""

query_job_cenario_pandemia = client.query(query_cenario_pandemia)

df_cenario_pandemia = query_job_cenario_pandemia.to_dataframe()

df_cenario_pandemia.head()

,mes_semana,casos,obitos
0,5/1,59380,3877
1,5/2,77203,5006
2,5/3,114107,6377
3,5/4,151191,6824
4,6/1,174406,7096


In [ ]:
df_cenario_pandemia.to_csv('df_cenario_pandemia.csv', index=False)

In [ ]:
fig = px.line(df_cenario_pandemia, x='mes_semana', y=['casos', 'obitos'], labels={'value': 'Pessoas', 'mes_semana': 'Mês/Semana'}, title='Casos e Óbitos por Mês/Semana')
fig.show()

In [ ]:
query_home_office = """
SELECT
  mes_semana,
  COUNT(trabalho_remoto) AS trabalho_remoto
FROM
  continual-apex-402217.covid.pnad_covid
GROUP BY mes_semana
ORDER BY mes_semana ASC
"""

query_job_home_office = client.query(query_home_office)

df_home_office = query_job_home_office.to_dataframe()

df_home_office.head()

,mes_semana,trabalho_remoto
0,5/1,16376
1,5/2,16668
2,5/3,17072
3,5/4,17396
4,6/1,17996


In [ ]:
df_home_office.to_csv('df_home_office.csv', index=False)

In [ ]:
fig = px.line(df_home_office, x='mes_semana', y='trabalho_remoto', labels={'value': 'Pessoas', 'mes_semana': 'Mês/Semana'}, title='Trabalho Remoto por Mês/Semana')
fig.update_traces(line=dict(color='blue', dash='dash'))

fig.show()

In [ ]:
df_covid = pd.merge(df_cenario_pandemia, df_home_office, on='mes_semana', how='left')

df_covid.head()

,mes_semana,casos,obitos,trabalho_remoto
0,5/1,59380,3877,16376
1,5/2,77203,5006,16668
2,5/3,114107,6377,17072
3,5/4,151191,6824,17396
4,6/1,174406,7096,17996


In [ ]:
df_covid.to_csv('df_covid.csv', index=False)

In [ ]:
fig = go.Figure()

# Adicionando as primeiras duas medidas no eixo Y principal
fig.add_trace(go.Scatter(x=df_covid['mes_semana'], y=df_covid['casos'], mode='lines', name='Casos', line=dict(color='orange')))

# Adicionando a terceira medida (trabalho_remoto) em um segundo eixo Y
fig.add_trace(go.Scatter(x=df_covid['mes_semana'], y=df_covid['trabalho_remoto'], mode='lines', name='Trabalho Remoto', line=dict(color='blue', dash='dash'), yaxis='y2'))

# Configurando os layouts dos eixos
fig.update_layout(
    title='Casos vs Trabalho Remoto',
    xaxis=dict(title='Mês/Semana'),
    yaxis=dict(title='Casos'),
    yaxis2=dict(title='Trabalho Remoto', overlaying='y', side='right')
)

fig.show()

In [ ]:
fig = go.Figure()

# Adicionando as primeiras duas medidas no eixo Y principal
fig.add_trace(go.Scatter(x=df_covid['mes_semana'], y=df_covid['obitos'], mode='lines', name='Óbitos', line=dict(color='red')))

# Adicionando a terceira medida (trabalho_remoto) em um segundo eixo Y
fig.add_trace(go.Scatter(x=df_covid['mes_semana'], y=df_covid['trabalho_remoto'], mode='lines', name='Trabalho Remoto', line=dict(color='blue', dash='dash'), yaxis='y2'))

# Configurando os layouts dos eixos
fig.update_layout(
    title='Óbitos vs Trabalho Remoto',
    xaxis=dict(title='Mês/Semana'),
    yaxis=dict(title='Óbitos'),
    yaxis2=dict(title='Trabalho Remoto', overlaying='y', side='right')
)

fig.show()

In [ ]:
fig = go.Figure()

# Adicionando as primeiras duas medidas no eixo Y principal
fig.add_trace(go.Scatter(x=df_covid['mes_semana'], y=df_covid['casos'], mode='lines', name='Casos', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=df_covid['mes_semana'], y=df_covid['obitos'], mode='lines', name='Óbitos', line=dict(color='red')))

# Adicionando a terceira medida (trabalho_remoto) em um segundo eixo Y
fig.add_trace(go.Scatter(x=df_covid['mes_semana'], y=df_covid['trabalho_remoto'], mode='lines', name='Trabalho Remoto', yaxis='y2', line=dict(color='blue', dash='dash')))

# Configurando os layouts dos eixos
fig.update_layout(
    title='Casos, Óbitos vs Trabalho Remoto',
    xaxis=dict(title='Mês/Semana'),
    yaxis=dict(title='Casos e Óbitos'),
    yaxis2=dict(title='Trabalho Remoto', overlaying='y', side='right')
)

fig.show()